In [2]:
!pip install evaluate rouge-score sentence-transformers > /dev/null 2>&1;

In [3]:
from datasets import load_dataset
from collections import defaultdict
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import evaluate
from sklearn.cluster import DBSCAN
from torch.utils.data import Dataset, DataLoader
import numpy as np
import random
import random
from transformers import BartForConditionalGeneration, BartTokenizer
from torch.optim import AdamW
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
rouge=evaluate.load("rouge")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
class RelRegModel(nn.Module):
    def __init__(self, model_name="bert-base-uncased"):
        super(RelRegModel, self).__init__()
        self.encoder = BertModel.from_pretrained(model_name)
        self.regressor = nn.Linear(self.encoder.config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.encoder(input_ids=input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        score = self.regressor(pooled_output)
        return score

In [6]:
model_relreg=torch.load('../input/anlp-relreg/RelReg.pth')
tokenizer_relreg = BertTokenizer.from_pretrained("bert-base-uncased")

/tmp/ipykernel_30/3652340851.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_relreg=torch.load('../input/anlp-relreg/RelReg.pth')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
model=model.to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
answersumm = load_dataset("alexfabbri/answersumm")

README.md:   0%|          | 0.00/9.74k [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/24.8M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/4.43M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/8.76M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2783 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
def compute_rouge_score(sent,ref_summary):
    results=rouge.compute(predictions=[sent], references=[ref_summary])
    return results

In [10]:
model_name = "facebook/bart-large"
tokenizer2 = BartTokenizer.from_pretrained(model_name)
model2=torch.load('../input/bart-ft2/BART_FT2.pth')
model2=model2.to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/tmp/ipykernel_30/1976436074.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2=torch.load('../input/bart-ft2/BART_FT2.pth')


In [11]:
def generate_summary(input_text):
    inputs = tokenizer2(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model2.generate(inputs["input_ids"].to(device), max_length=256, min_length=10, length_penalty=2.0, num_beams=4)
    summary = tokenizer2.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [13]:
test_data=answersumm['test']
final_sums=[]
final_summ_scores=[]
true_sums=[]
for sample in tqdm(test_data):
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    sents=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            sents.append(sent['text'])
    random.shuffle(sents)
    final_summ=generate_summary(' '.join([x for x in sents]))
    final_sums.append(final_summ)
    true_sums.append(ref_summ)
    final_summ_scores.append(compute_rouge_score(final_summ,ref_summ))
    
sum_dict = defaultdict(int)
count_dict = defaultdict(int)
for d in final_summ_scores:
    for key, value in d.items():
        sum_dict[key] += value
        count_dict[key] += 1
average_dict = {key: sum_val / count_dict[key] for key, sum_val in sum_dict.items()}
average_dict

100%|██████████| 1000/1000 [12:30<00:00,  1.33it/s]


{'rouge1': 0.24676048878644827,
 'rouge2': 0.07045343781907064,
 'rougeL': 0.18139636217125518,
 'rougeLsum': 0.1814796955045885}

In [ ]:
test_data=answersumm['test']
final_sums=[]
final_summ_scores=[]
for sample in tqdm(test_data):
    test_dict={}
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    q_emb=model.encode(query,show_progress_bar=False)
    sent_lst=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            sent_lst.append(sent['text'])
    sent_emb=model.encode(sent_lst,show_progress_bar=False)
    for sen,sen_emb in zip(sent_lst,sent_emb):
        test_dict[tuple(sen_emb)]={'sentence':sen, 'query': query, 'q_emb': q_emb}
    # DBSCAN
    data=list(test_dict.keys())
    dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
    labels = dbscan.fit_predict(data)
    grouped = defaultdict(list)
    for emb, clust_id in zip(data, labels):
        if clust_id!=-1:  
            grouped[clust_id].append(test_dict[emb]['sentence'])
    # within group ranking and summ
    result = list(grouped.values())
    group_summaries=[]
    for group in result:
        sent_scores={}
        for sent in group:
            inputs = tokenizer_relreg(query, sent, 
                                max_length=512, 
                                padding="max_length", 
                                truncation=True, 
                                return_tensors="pt")
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            token_type_ids = inputs['token_type_ids'].to(device)
            sent_score = model_relreg(input_ids, attention_mask, token_type_ids).item()
            sent_scores[sent]=sent_score
        sorted_dict = sorted(sent_scores.items(), key=lambda item: item[1], reverse=True)
        top_5_sents = ' '.join([key for key, _ in sorted_dict[:3]])
        group_summaries.append(top_5_sents)
    final_summ=generate_summary(' '.join([x for x in group_summaries]))
    final_sums.append(final_summ)
    final_summ_scores.append(compute_rouge_score(final_summ,ref_summ))

sum_dict = defaultdict(int)
count_dict = defaultdict(int)
for d in final_summ_scores:
    for key, value in d.items():
        sum_dict[key] += value
        count_dict[key] += 1
average_dict = {key: sum_val / count_dict[key] for key, sum_val in sum_dict.items()}
average_dict

100%|██████████| 1000/1000 [21:30<00:00,  1.29s/it]


{'rouge1': 0.21996755451800554,
 'rouge2': 0.05378383290463431,
 'rougeL': 0.16190530965770017,
 'rougeLsum': 0.16196501115023748}

In [ ]:
test_data=answersumm['test']
final_sums=[]
final_summ_scores=[]
for sample in tqdm(test_data):
    test_dict={}
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    q_emb=model.encode(query,show_progress_bar=False)
    sent_lst=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            sent_lst.append(sent['text'])
    sent_emb=model.encode(sent_lst,show_progress_bar=False)
    for sen,sen_emb in zip(sent_lst,sent_emb):
        test_dict[tuple(sen_emb)]={'sentence':sen, 'query': query, 'q_emb': q_emb}
    # DBSCAN
    data=list(test_dict.keys())
    dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
    labels = dbscan.fit_predict(data)
    grouped = defaultdict(list)
    for emb, clust_id in zip(data, labels):
        if clust_id!=-1:  
            grouped[clust_id].append(test_dict[emb]['sentence'])
    # within group ranking and summ
    result = list(grouped.values())
    group_summaries=[]
    for group in result:
        sent_scores={}
        for sent in group:
            inputs = tokenizer_relreg(query, sent, 
                                max_length=512, 
                                padding="max_length", 
                                truncation=True, 
                                return_tensors="pt")
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)
            token_type_ids = inputs['token_type_ids'].to(device)
            sent_score = model_relreg(input_ids, attention_mask, token_type_ids).item()
            sent_scores[sent]=sent_score
        sorted_dict = sorted(sent_scores.items(), key=lambda item: item[1], reverse=True)
        top_5_sents = ' '.join([key for key, _ in sorted_dict[:7]])
        group_summaries.append(top_5_sents)
    final_summ=generate_summary(' '.join([x for x in group_summaries]))
    final_sums.append(final_summ)
    final_summ_scores.append(compute_rouge_score(final_summ,ref_summ))

sum_dict = defaultdict(int)
count_dict = defaultdict(int)
for d in final_summ_scores:
    for key, value in d.items():
        sum_dict[key] += value
        count_dict[key] += 1
average_dict = {key: sum_val / count_dict[key] for key, sum_val in sum_dict.items()}
average_dict

100%|██████████| 1000/1000 [22:33<00:00,  1.35s/it]


{'rouge1': 0.23835263676056226,
 'rouge2': 0.0653611800576848,
 'rougeL': 0.17550960634175589,
 'rougeLsum': 0.1756511992621099}

In [29]:
test_data=answersumm['test']
final_sums=[]
final_summ_scores=[]
for sample in tqdm(test_data):
    test_dict={}
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    q_emb=model.encode(query,show_progress_bar=False)
    sent_lst=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            sent_lst.append(sent['text'])
    sent_emb=model.encode(sent_lst,show_progress_bar=False)
    for sen,sen_emb in zip(sent_lst,sent_emb):
        test_dict[tuple(sen_emb)]={'sentence':sen, 'query': query, 'q_emb': q_emb}
    # DBSCAN
    data=list(test_dict.keys())
    dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
    labels = dbscan.fit_predict(data)
    grouped = defaultdict(list)
    for emb, clust_id in zip(data, labels):
        if clust_id!=-1:  
            grouped[clust_id].append(test_dict[emb]['sentence'])
    # within group ranking and summ
    result = list(grouped.values())
    group_summaries=[]
    for group in result:
        sent_scores={}
        for sent in group:
            sentence_embedding = model.encode(sent,show_progress_bar=False)
            sent_score = (cosine_similarity(q_emb.reshape(1, -1),sentence_embedding.reshape(1,-1)).tolist())[0]
            sent_scores[sent]=sent_score
        sorted_dict = sorted(sent_scores.items(), key=lambda item: item[1], reverse=True)
        top_5_sents = ' '.join([key for key, _ in sorted_dict[:5]])
        group_summaries.append(top_5_sents)
    final_summ=generate_summary(' '.join([x for x in group_summaries]))
    final_sums.append(final_summ)
    final_summ_scores.append(compute_rouge_score(final_summ,ref_summ))

sum_dict = defaultdict(int)
count_dict = defaultdict(int)
for d in final_summ_scores:
    for key, value in d.items():
        sum_dict[key] += value
        count_dict[key] += 1
average_dict = {key: sum_val / count_dict[key] for key, sum_val in sum_dict.items()}
average_dict

100%|██████████| 1000/1000 [18:34<00:00,  1.11s/it]


{'rouge1': 0.24211679479813913,
 'rouge2': 0.06914819376243228,
 'rougeL': 0.17978250493986755,
 'rougeLsum': 0.17984311100047362}

In [ ]:
test_data=answersumm['test']
final_sums=[]
final_summ_scores=[]
for sample in tqdm(test_data):
    test_dict={}
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    q_emb=model.encode(query,show_progress_bar=False)
    sent_lst=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            sent_lst.append(sent['text'])
    sent_emb=model.encode(sent_lst,show_progress_bar=False)
    for sen,sen_emb in zip(sent_lst,sent_emb):
        test_dict[tuple(sen_emb)]={'sentence':sen, 'query': query, 'q_emb': q_emb}
    # DBSCAN
    data=list(test_dict.keys())
    dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
    labels = dbscan.fit_predict(data)
    grouped = defaultdict(list)
    for emb, clust_id in zip(data, labels):
        if clust_id!=-1:  
            grouped[clust_id].append(test_dict[emb]['sentence'])
    # within group ranking and summ
    result = list(grouped.values())
    group_summaries=[]
    for group in result:
        sent_scores={}
        for sent in group:
            sentence_embedding = model.encode(sent,show_progress_bar=False)
            sent_score = (cosine_similarity(q_emb.reshape(1, -1),sentence_embedding.reshape(1,-1)).tolist())[0]
            sent_scores[sent]=sent_score
        sorted_dict = sorted(sent_scores.items(), key=lambda item: item[1], reverse=True)
        top_5_sents = ' '.join([key for key, _ in sorted_dict[:3]])
        group_summaries.append(top_5_sents)
    final_summ=generate_summary(' '.join([x for x in group_summaries]))
    final_sums.append(final_summ)
    final_summ_scores.append(compute_rouge_score(final_summ,ref_summ))

sum_dict = defaultdict(int)
count_dict = defaultdict(int)
for d in final_summ_scores:
    for key, value in d.items():
        sum_dict[key] += value
        count_dict[key] += 1
average_dict = {key: sum_val / count_dict[key] for key, sum_val in sum_dict.items()}
average_dict

In [11]:
test_data=answersumm['test']
final_sums=[]
final_summ_scores=[]
for sample in tqdm(test_data):
    test_dict={}
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    q_emb=model.encode(query,show_progress_bar=False)
    sent_lst=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            sent_lst.append(sent['text'])
    sent_emb=model.encode(sent_lst,show_progress_bar=False)
    for sen,sen_emb in zip(sent_lst,sent_emb):
        test_dict[tuple(sen_emb)]={'sentence':sen, 'query': query, 'q_emb': q_emb}
    # DBSCAN
    data=list(test_dict.keys())
    dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
    labels = dbscan.fit_predict(data)
    grouped = defaultdict(list)
    for emb, clust_id in zip(data, labels):
        if clust_id!=-1:  
            grouped[clust_id].append(test_dict[emb]['sentence'])
    # within group ranking and summ
    result = list(grouped.values())
    group_summaries=[]
    for group in result:
        sent_scores={}
        for sent in group:
            sentence_embedding = model.encode(sent,show_progress_bar=False)
            sent_score = (cosine_similarity(q_emb.reshape(1, -1),sentence_embedding.reshape(1,-1)).tolist())[0]
            sent_scores[sent]=sent_score
        sorted_dict = sorted(sent_scores.items(), key=lambda item: item[1], reverse=True)
        top_5_sents = ' '.join([key for key, _ in sorted_dict])
        group_summaries.append(top_5_sents)
    final_summ=generate_summary(' '.join([x for x in group_summaries]))
    final_sums.append(final_summ)
    final_summ_scores.append(compute_rouge_score(final_summ,ref_summ))

sum_dict = defaultdict(int)
count_dict = defaultdict(int)
for d in final_summ_scores:
    for key, value in d.items():
        sum_dict[key] += value
        count_dict[key] += 1
average_dict = {key: sum_val / count_dict[key] for key, sum_val in sum_dict.items()}
average_dict

100%|██████████| 1000/1000 [19:45<00:00,  1.19s/it]


{'rouge1': 0.25354954841498867,
 'rouge2': 0.07353783573612624,
 'rougeL': 0.1861692653576872,
 'rougeLsum': 0.18621959774135294}

In [84]:
ref_summ

'There are a number of ways to succeed taunt kills that is connected to choosing the right targets, distraction and prediction of their moves. Snipers seem to be the easiest target.'

In [85]:
final_summ

"They will focus down their scope for days, completely oblivious to the world around them, feeling perfectly safe wearing their razorbacks.Even huntsmen snipers tend to focus just before a shot.Pick your target well.If you're being chased, and you think you can time it right, round a corner, wait if necessary, then perform the taunt.If your goal is specifically to get a taunt kill, you may want to favor certain classes.You are not likely to be able to"

In [96]:
for sample in test_data.select(range(99,100)):
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    inp_str=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            inp_str.append(sent['text'])
    random.shuffle(inp_str)
    inp_str=' '.join(inp_str)
    inputs = tokenizer2(inp_str,
            max_length=1024,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            )
    with torch.no_grad():
        output_ids = model2.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                max_length=100,
                num_beams=4,
                early_stopping=True,
        )
    s=[tokenizer2.decode(ids, skip_special_tokens=True) for ids in output_ids]
    print(s)
compute_rouge_score(s[0],ref_summ)

["Playing as the Heavy with the Holiday Punch gloves equipped, you can hit enemies from behind to make the enemy laugh. While they're laughing you can easily taunt-kill them If you are playing as a Heavy, you should be able to taunt kill most enemies. There are two relatively easy ways to do this, that works with most taunt kills. Snipers. Sneak up behind them, preferably a sniper as they tend not to move too much, and give em a good whack."]


{'rouge1': 0.2608695652173913,
 'rouge2': 0.05309734513274336,
 'rougeL': 0.13913043478260867,
 'rougeLsum': 0.13913043478260867}

In [67]:
# for x in final_sums:
#     print(x)
#     print()
c=0
for i,x in enumerate(zip(final_sums,final_summ_scores)):
    if x[1]['rougeL']<0.15:
        c+=1
        print(i)
        print(answersumm['test'][i]['summaries'][0][1])
        print()
        print(x[0],x[1])
        print()
        break
c

0
Demomen are best against a sentry farm. Heavies or pyros can also be effective. The medic should lead the uber combo. If possible, back the uber team(s) up with as much raw firepower as you can muster. Go with corner-edging soldiers and sneaky demos.  The Machina can help damage Engies that hide behind their sentry, but for unattended sentries, the Sydney Sleeper charges up very fast.  The best way you can help your team is by building up a teleporter, so that they can get to and take down the other team's Sentry.    

In reponse to a question about the Turret Song on Reddit , the author of the song "The Potatos Lament" stated as follows:In reply to the same question, the author answered:I can hear 'lacrimosa' at the beginning of the Lament.Translate it, you get 'Crying or Tearful Potato".Potato Lacrimosa (Weeping Potato) Potato Po Uota (Power Potato Vows) Diva MePotato Lament (Lament) Potato Lamentation (Crying Potato) 'Potato' to be exact.As for the lyrics, she's probably singing i

1

In [9]:
data=list(test_dict.keys())
dbscan = DBSCAN(eps=0.5, min_samples=2, metric='cosine')
labels = dbscan.fit_predict(data)
print("Cluster Labels:", labels)

Cluster Labels: [ 0  1  1  1 -1 -1  1 -1  1  0  1  1 -1 -1 -1  1 -1  1  1 -1  1 -1  1 -1
 -1  1 -1]


In [35]:
grouped = defaultdict(list)
for emb, clust_id in zip(data, labels):
    grouped[clust_id].append(test_dict[emb]['sentence'])
grouped[1]

["I've tried fixing an Xbox 360 myself by doing the X-Clamp replacement but that fix only lasted about 8 months before it red ringed again.",
 'Since Microsoft paid 1.5 billion dollars for you to have a 3 year warranty for your Xbox 360, you should just have them fix it for you for free.',
 "That is, assuming you've had this Xbox 360 less than 3 years.",
 "Your 360 might run for about 10 minutes, and you've just voided any warranty you may have had.",
 'I would not advise opening your Xbox 360 to fix the RRoD. Unless a lot has changed in a few years, the fixes I found for RRoD only made the problem worse.',
 'I was personally never able to get my RRoD console fixed with solutions found on the internet.',
 'And then I found that once I had opened it, Microsoft would not even touch the console for any amount of money.',
 'Worked for me when I got the RRoD',
 "As an owner of a video games repair shop, the kits do work only for certain fixes, but they definitely won't last as long as a ref

In [11]:
answersumm['test'][1]['summaries'][0][1]

' It voids your warranty if you open up the console. Microsoft give a 3 year warranty with Xbox 360, you should just have them fix it for you for free. I would not advise opening your Xbox 360 to fix the RRoD. Unless a lot has changed in a few years, the fixes for RRoD only make the problem worse.'

In [12]:
answersumm['test'][1]['question']['question']

"I've heard of kits that can fix your Xbox 360 if you happen to get the  Red Ring of Death . Do any of these work, and do they void your warranty? Are there any other noteworthy tips or tricks?"

In [14]:
class RelRegModel(nn.Module):
    def __init__(self, model_name="bert-base-uncased"):
        super(RelRegModel, self).__init__()
        self.encoder = BertModel.from_pretrained(model_name)
        self.regressor = nn.Linear(self.encoder.config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.encoder(input_ids=input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids)
        pooled_output = outputs.pooler_output
        score = self.regressor(pooled_output)
        return score

In [ ]:
model_relreg=torch.load('../input/anlp-relreg/RelReg.pth')
tokenizer_relreg = BertTokenizer.from_pretrained("bert-base-uncased")

In [37]:
curr_data=answersumm['test']
sample_dict={}
for sample in tqdm(curr_data.select(range(1,2))):
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    sents=[]
    temp_scores=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            inputs = tokenizer_relreg(query, sent['text'], 
                            max_length=512, 
                            padding="max_length", 
                            truncation=True, 
                            return_tensors="pt")
            sents.append(sent['text'])
            curr={
                'input_ids': inputs['input_ids'],
                'attention_mask': inputs['attention_mask'],
                'token_type_ids': inputs['token_type_ids'],
            }

            input_ids = curr['input_ids'].to(device)
            attention_mask = curr['attention_mask'].to(device)
            token_type_ids = curr['token_type_ids'].to(device)
            outputs = model_relreg(input_ids, attention_mask, token_type_ids)
            sample_dict[sent['text']]=outputs.item()
sample_dict=sorted(sample_dict.items(), key=lambda item: item[1], reverse=True)

100%|██████████| 1/1 [00:00<00:00,  1.66it/s]


In [38]:
sample_dict

[('Since Microsoft paid 1.5 billion dollars for you to have a 3 year warranty for your Xbox 360, you should just have them fix it for you for free.',
  0.19985000789165497),
 ('Consider the kits as temporary fixes for some red ring of death related issues.',
  0.18345361948013306),
 ("Your 360 might run for about 10 minutes, and you've just voided any warranty you may have had.",
  0.18208950757980347),
 ('It voids your warranty if you open up the console.', 0.16600573062896729),
 ('You may hear of something called the towel trick if you have googled this.',
  0.1521153450012207),
 ('I would not advise opening your Xbox 360 to fix the RRoD. Unless a lot has changed in a few years, the fixes I found for RRoD only made the problem worse.',
  0.1493116319179535),
 ("As an owner of a video games repair shop, the kits do work only for certain fixes, but they definitely won't last as long as a reflow of the board.",
  0.14652936160564423),
 ('Let alone it voids your warranty.', 0.14080893993

In [21]:
model_name = "facebook/bart-large"
tokenizer2 = BartTokenizer.from_pretrained(model_name)
model2 = BartForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [76]:
def generate_summary(input_text):
    inputs = tokenizer2(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model2.generate(inputs["input_ids"].to(device), max_length=100, min_length=10, length_penalty=2.0, num_beams=4)
    summary = tokenizer2.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [36]:
curr_data=answersumm['test']
scores_5=[]
scores_10=[]
scores_15=[]
for sample in tqdm(curr_data.select(range(1,2))):
    ref_summ=sample['summaries'][0][1]
    query=sample['question']['question']
    sents=[]
    temp_scores=[]
    for ans in sample['answers']:
        for sent in ans['sents']:
            inputs = tokenizer_relreg(query, sent['text'], 
                            max_length=512, 
                            padding="max_length", 
                            truncation=True, 
                            return_tensors="pt")
            sents.append(sent['text'])
            curr={
                'input_ids': inputs['input_ids'],
                'attention_mask': inputs['attention_mask'],
                'token_type_ids': inputs['token_type_ids'],
            }

            input_ids = curr['input_ids'].to(device)
            attention_mask = curr['attention_mask'].to(device)
            token_type_ids = curr['token_type_ids'].to(device)
            outputs = model_relreg(input_ids, attention_mask, token_type_ids)
            temp_scores.extend(outputs.detach().cpu().numpy().flatten().tolist())

    sorted_strings = [string for _, string in sorted(zip(temp_scores, sents), reverse=True)]
    inp_str=""
    for s in sorted_strings[:5]:
        inp_str+=s
    temp=generate_summary(inp_str)
    scores_5.append(compute_rouge_score(temp,ref_summ))
    print(temp)
    inp_str=""
    for s in sorted_strings[:10]:
        inp_str+=s
    temp=generate_summary(inp_str)
    scores_10.append(compute_rouge_score(temp,ref_summ))
    print(temp)
    inp_str=""
    for s in sorted_strings[:15]:
        inp_str+=s
    temp=generate_summary(inp_str)
    scores_15.append(compute_rouge_score(temp,ref_summ))
    print(temp)

  0%|          | 0/1 [00:00<?, ?it/s]

Consider the kits as temporary fixes for some red ring of death related issues.You may hear of something called the towel trick if you have googled this.It voids your warranty if you open up the console.Your 360 might run for about 10 minutes, and you've just voided any warranty you may have had.Since Microsoft paid 1.5 billion dollars for you to have a 3 year warranty for your Xbox 360, you should just have them fix it for you for free.
I don't have any experience with the kits you're looking at, but try unplugging your AC adaptor, dusting it out, moving it to a spot with more ventilation, and plugging it back in.Your 360 might run for about 10 minutes, and you've just voided any warranty you may have had.As an owner of a video games repair shop, the kits do work only for certain fixes, but they definitely won't last as long as a reflow of the board.Consider the kits as temporary fixes for some red ring of death related issues. Unless a lot has changed in a few years, the fixes I foun

100%|██████████| 1/1 [00:07<00:00,  8.00s/it]

I know many people do their own research and find forums regarding this RROD repair topic, but I must honestly say there is a lot of false and misleading information from others that have fixed their own systems or only a handful of them.I've tried fixing an Xbox 360 myself by doing the X-Clamp replacement but that fix only lasted about 8 months before it red ringed again. Unless a lot has changed in a few years, the fixes I found for RRoD only made the problem worse.I don't have any experience with the kits you're looking at, but try unplugging your AC adaptor, dusting it out, moving it to a spot with more ventilation, and plugging it back in.Your 360 might run for about 10 minutes, and you've just voided any warranty you may have had.Consider the kits as temporary fixes for some red ring of death related issues.As an owner of a video games repair shop, the kits do


In [30]:
scores_5,scores_10,scores_15

([{'rouge1': 0.4761904761904762,
   'rouge2': 0.3586206896551724,
   'rougeL': 0.380952380952381,
   'rougeLsum': 0.380952380952381}],
 [{'rouge1': 0.44347826086956527,
   'rouge2': 0.3508771929824561,
   'rougeL': 0.22608695652173916,
   'rougeLsum': 0.22608695652173916}],
 [{'rouge1': 0.35497835497835495,
   'rouge2': 0.13973799126637554,
   'rougeL': 0.20779220779220778,
   'rougeLsum': 0.20779220779220778}])